In [1]:
print("welcome to notebook")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1689268833492_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

welcome to notebook

In [2]:
sc.install_pypi_package("Cython")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package("pandas==1.0.5")
sc.install_pypi_package("matplotlib==3.2.2", "https://pypi.org/simple")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package("seaborn")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc, asc, when, datediff
from pyspark.sql.types import StringType , IntegerType
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import col, split, countDistinct
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql import Window
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, RandomForestClassifier, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
from pyspark.ml.feature import StringIndexer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler, IndexToString
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
from pyspark.sql.types import *
from pyspark.sql.types import FloatType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql import SparkSession

## Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

## Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [7]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']

In [8]:
def clean_data(df):

    df_clean = df.filter(df["firstName"].isNotNull()).dropDuplicates()
    return df_clean

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
a_check = clean_data(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
a_check.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']

In [11]:
#a_check.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def prepare_dataset(df):
    
    # Clean dataset using clean_data function
    df = clean_data(df)
    
    get_date = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%m-%d-%Y"))

    df = df.withColumn("ts_date", to_date(get_date(df["ts"]), "MM-dd-yyyy"))
    df = df.withColumn("registration_date", to_date(get_date(df["registration"]), "MM-dd-yyyy"))
    
    # Create column Churn when the event recorded is 'Cancellation Confirmation'. 
    # A value equal to 1 indicates that the user cancelled the subscription.
    churn_cancellation = udf(lambda x: 1 if x=="Cancellation Confirmation" else 0, IntegerType())
    
    df = df.withColumn("churn_cancellation", churn_cancellation("page"))
    
    # Convert column gender to numeric: 1 for 'female' and 0 for 'male'
    gender = udf(lambda x: 1 if x=="F" else 0, IntegerType())
    
    df = df.withColumn("gender", gender("gender"))
    
    
    # Compute active days as number of days since registration
    # Create dataframe with last timestamp (ts_date) per user
    df_last_ts = df.groupBy('userId').agg(max('ts_date').alias('last_interaction'))
    
    df = df.join(df_last_ts, on='userId').select(df_last_ts['*'], df['*']).withColumn('days_registered', datediff(df_last_ts['last_interaction'], df['registration_date']).cast('float'))
    
    # Create a new column 'date' with format 'yyyy-MM-dd'
    df = df.withColumn('date', date_format('ts_date', 'yyyy-MM-dd').alias('date'))

    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# bcheck = prepare_dataset(df)
# bcheck.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def build_features(df):
    
    df = prepare_dataset(df)
    # Compute average songs played by day per user
    avg_songs = df.filter(df.page == "NextSong").groupBy('userId',"ts_date").agg({'page':'count'})\
                                        .groupBy("userId").mean().withColumnRenamed("avg(count(page))","avg_songs")\
                                        .withColumn("avg_songs", round("avg_songs", 2))
    avg_songs = avg_songs.select(avg_songs['userId'].alias('songs_userId'), "avg_songs")
    #avg_songs.show()
    
    # Compute the number of thumbs up for user
    thumbs_up = df.filter(df.page == "Thumbs Up").groupBy('userId').agg({'page':'count'})
    thumbs_up = thumbs_up.select(thumbs_up['userId'].alias('thumbsup_userId'), thumbs_up['count(page)'].alias('thumbs_up_count'))
    #thumbs_up.show()
    
    # Compute the number of thumbs down per user
    thumbs_down = df.filter(df.page == "Thumbs Down").groupBy('userId').agg({'page':'count'})
    thumbs_down = thumbs_down.select(thumbs_down['userId'].alias('thumbsdown_userId'), thumbs_down['count(page)'].alias('thumbs_down_count'))
      

    # Compute the number of add friend events per user
    add_friend = df.filter(df.page == "Add Friend").groupBy('userId').agg({'page':'count'})\
                                     .withColumnRenamed("count(page)","num_friends")
    add_friend = add_friend.select(add_friend['userId'].alias('add_friend_userId'), "num_friends")
    
    # Compute the fraction of page 'roll advert'
    avg_roll_adv = df.filter(df.page == "Roll Advert").groupBy('userId',"ts_date").agg({'page':'count'})\
                                        .groupBy("userId").mean().withColumnRenamed("avg(count(page))","avg_roll_advert")\
                                        .withColumn("avg_roll_advert", round("avg_roll_advert", 2))
    avg_roll_adv = avg_roll_adv.select(avg_roll_adv['userId'].alias('roll_adv_userId'), "avg_roll_advert")
    
    # daily help
    dailyHelpVisit = df.filter(df.page == "Help").groupBy('userId',"ts_date").agg({'page':'count'})\
                                        .groupBy("userId").mean().withColumnRenamed("avg(count(page))","help_visit")
    dailyHelpVisit = dailyHelpVisit.select(dailyHelpVisit['userId'].alias('help_userId'), "help_visit")
    
    #daily error
    dailyErrorVisit = df.filter(df.page == "Error").groupBy('userId',"ts_date").agg({'page':'count'})\
                                        .groupBy("userId").mean().withColumnRenamed("avg(count(page))","error_visit")
    
    dailyErrorVisit = dailyErrorVisit.select(dailyErrorVisit['userId'].alias('error_userId'), "error_visit")                                 
    #total playlist
    num_playlist = df.filter(df.page == "Add to Playlist").groupBy('userId').agg({'page':'count'})\
                                        .withColumnRenamed("count(page)","total_playlist")\
                                        .withColumn("total_playlist", round("total_playlist", 2))
    num_playlist = num_playlist.select(num_playlist['userId'].alias('playlist_userId'), "total_playlist")
                  
    #print(df.columns)
    level = udf(lambda x: 1 if x=="paid" else 0, IntegerType())
    levels = df.select(['userId', 'level', 'ts_date'])\
                        .orderBy(desc('ts_date'))\
                        .dropDuplicates(['userId'])\
                        .select(['userId', 'level'])\
                        .withColumn('last_level', level(df['level']).cast(IntegerType()))
    levels = levels.drop('level')
    levels = levels.withColumnRenamed('userId', 'level_userId')
    
    # Combine all the features into a single dataframe
    df_ml = df.select('userId', 'gender', 'churn_cancellation', 'days_registered').dropna().drop_duplicates() \
              .join(avg_songs, avg_songs.songs_userId == df.userId, how='left')\
              .join(levels, levels.level_userId == df.userId, how='left')\
              .join(thumbs_up, thumbs_up.thumbsup_userId == df.userId, how='left') \
              .join(thumbs_down, thumbs_down.thumbsdown_userId == df.userId, how='left') \
              .join(add_friend, add_friend.add_friend_userId == df.userId, how='left') \
              .join(avg_roll_adv, avg_roll_adv.roll_adv_userId == df.userId, how='left') \
              .join(dailyHelpVisit, dailyHelpVisit.help_userId == df.userId, how='left') \
              .join(dailyErrorVisit, dailyErrorVisit.error_userId == df.userId, how='left') \
              .join(num_playlist, num_playlist.playlist_userId == df.userId, how='left') \
              .drop('songs_userId', 'thumbsup_userId', 'thumbsdown_userId', 'add_friend_userId', 'roll_adv_userId', 'help_userId', 'error_userId', 'playlist_userId',"level_userId") \
              .fillna(0)
    
    return df_ml


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_model=build_features(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_model.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['userId', 'gender', 'churn_cancellation', 'days_registered', 'avg_songs', 'last_level', 'thumbs_up_count', 'thumbs_down_count', 'num_friends', 'avg_roll_advert', 'help_visit', 'error_visit', 'total_playlist']

In [17]:
df_model.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o528.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 4 times, most recent failure: Lost task 0.3 in stage 7.0 (TID 222) (ip-172-31-39-58.us-east-2.compute.internal executor 1): java.lang.RuntimeException: Failed to run command: /usr/bin/virtualenv -p python3 --no-pip --system-site-packages virtualenv_application_1689268833492_0001_0
	at org.apache.spark.api.python.VirtualEnvFactory.execCommand(VirtualEnvFactory.scala:125)
	at org.apache.spark.api.python.VirtualEnvFactory.setupVirtualEnv(VirtualEnvFactory.scala:83)
	at org.apache.spark.api.python.PythonWorkerFactory.<init>(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.$anonfun$createPythonWorker$1(SparkEnv.scala:128)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:128)
	at org.apache.spark.api.python.BasePythonRunner.comp

In [18]:
#df_model.select("gender").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Train-test-split
train, test = df_model.randomSplit([0.8, 0.2], seed=23)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
train.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o535.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 4 times, most recent failure: Lost task 0.3 in stage 18.0 (TID 342) (ip-172-31-39-58.us-east-2.compute.internal executor 1): java.lang.RuntimeException: Failed to run command: /usr/bin/virtualenv -p python3 --no-pip --system-site-packages virtualenv_application_1689268833492_0001_0
	at org.apache.spark.api.python.VirtualEnvFactory.execCommand(VirtualEnvFactory.scala:125)
	at org.apache.spark.api.python.VirtualEnvFactory.setupVirtualEnv(VirtualEnvFactory.scala:83)
	at org.apache.spark.api.python.PythonWorkerFactory.<init>(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.$anonfun$createPythonWorker$1(SparkEnv.scala:128)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:128)
	at org.apache.spark.api.python.BasePythonRunner.co

In [21]:
print(train.count(), test.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o535.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 29.0 failed 4 times, most recent failure: Lost task 1.3 in stage 29.0 (TID 454) (ip-172-31-42-63.us-east-2.compute.internal executor 3): java.lang.RuntimeException: Failed to run command: /usr/bin/virtualenv -p python3 --no-pip --system-site-packages virtualenv_application_1689268833492_0001_0
	at org.apache.spark.api.python.VirtualEnvFactory.execCommand(VirtualEnvFactory.scala:125)
	at org.apache.spark.api.python.VirtualEnvFactory.setupVirtualEnv(VirtualEnvFactory.scala:83)
	at org.apache.spark.api.python.PythonWorkerFactory.<init>(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.$anonfun$createPythonWorker$1(SparkEnv.scala:128)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:128)
	at org.apache.spark.api.python.BasePythonRunner.compute

In [22]:
def create_pipeline(df):
    # Assuming your DataFrame is called 'df' and the categorical column is 'states'

    # Step 1: StringIndexer to convert categorical column to numerical indices
    #stringIndexer = StringIndexer(inputCol="last_state", outputCol="states_indexed")

    # Step 2: OneHotEncoder to convert the indexed column to binary vectors
    #oneHotEncoder = OneHotEncoder(inputCol="states_indexed", outputCol="states_encoded")

    # Step 3: VectorAssembler to combine the encoded column with other features
    vectorAssembler = VectorAssembler(inputCols=["days_registered",
                                                 "avg_songs",
                                                 "last_level",
                                                 "thumbs_up_count",
                                                 "thumbs_down_count",
                                                 "num_friends",
                                                 "avg_roll_advert",
                                                 "help_visit",
                                                 "error_visit",
                                                 "total_playlist",
                                                  "gender"],
                                       outputCol="features")

    # Step 4: Normalizer to normalize the feature vector
    normalizer = Normalizer(inputCol="features", outputCol="normalized_features")

    # Create the pipeline
    #pipeline = Pipeline(stages=[stringIndexer, oneHotEncoder, vectorAssembler, normalizer])
    pipeline = Pipeline(stages=[vectorAssembler, normalizer])

    # Fit the pipeline to the DataFrame
    pipelineModel = pipeline.fit(df)

    # Transform the DataFrame
    transformed_df = pipelineModel.transform(df)
    #print(transformed_df.columns)
    # Show the result
    return transformed_df, pipelineModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
pipe_data, pipe_Model= create_pipeline(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
pipe_data.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------------------+---------------+---------+----------+---------------+-----------------+-----------+---------------+------------------+-----------+--------------+--------------------+--------------------+
| userId|gender|churn_cancellation|days_registered|avg_songs|last_level|thumbs_up_count|thumbs_down_count|num_friends|avg_roll_advert|        help_visit|error_visit|total_playlist|            features| normalized_features|
+-------+------+------------------+---------------+---------+----------+---------------+-----------------+-----------+---------------+------------------+-----------+--------------+--------------------+--------------------+
|1004060|     0|                 0|           59.0|   109.62|         0|            119|               30|         54|           1.75|1.6923076923076923|        1.0|            75|[59.0,109.62,0.0,...|[0.29835113847837...|
+-------+------+------------------+---------------+---------+----------+---------------+-----------------+--

In [25]:
trainingTransformed = pipe_Model.transform(train)
testTransformed = pipe_Model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
1+1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [27]:
# trainingTransformed.persist()
# trainingTransformed.take(1)
# testTransformed.persist()
# testTransformed.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
trainingTransformed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: string, gender: int, churn_cancellation: int, days_registered: float, avg_songs: double, last_level: int, thumbs_up_count: bigint, thumbs_down_count: bigint, num_friends: bigint, avg_roll_advert: double, help_visit: double, error_visit: double, total_playlist: bigint, features: vector, normalized_features: vector]

In [47]:
trainingTransformed.select("churn_cancellation").groupby("churn_cancellation").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+
|churn_cancellation|count|
+------------------+-----+
|                 1| 4009|
|                 0|17821|
+------------------+-----+

In [30]:
def ModelEvaluator(dfPredictions, label = 'churn_cancellation'):

    # Convert prediction and label columns to DoubleType
    colSelect = dfPredictions.select(
        [F.col('prediction').cast(DoubleType()),
         F.col(label).cast(DoubleType()).alias(label)])

    # Calculate accuracy
    evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol='prediction', metricName='accuracy')
    accuracy = evaluator.evaluate(colSelect)

    # Calculate precision, recall, and F1-score
    evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol='prediction', metricName='weightedPrecision')
    precision = evaluator.evaluate(colSelect)
    evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol='prediction', metricName='weightedRecall')
    recall = evaluator.evaluate(colSelect)
    evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol='prediction', metricName='f1')
    f1 = evaluator.evaluate(colSelect)

    # Calculate AUC
    evaluator = BinaryClassificationEvaluator(labelCol=label, rawPredictionCol='prediction', metricName='areaUnderROC')
    auc = evaluator.evaluate(dfPredictions)

    # Calculate precision-recall curve
    evaluator = BinaryClassificationEvaluator(labelCol=label, rawPredictionCol='prediction', metricName='areaUnderPR')
    pr_curve = evaluator.evaluate(dfPredictions)

#     # Calculate classification report
#     evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol='prediction', metricName='truePositiveRate')
#     tpr = evaluator.evaluate(colSelect)
#     evaluator = MulticlassClassificationEvaluator(labelCol=label, predictionCol='prediction', metricName='falsePositiveRate')
#     fpr = evaluator.evaluate(colSelect)
#     classification_report = {'weightedTruePositiveRate': tpr, 'weightedFalsePositiveRate': fpr}

    
#     preds_and_labels = dfPredictions.select(['prediction',label]).withColumn(label, col(label).cast(FloatType())).orderBy('prediction')
#     metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
    
#     confusion_matrix = metrics.confusionMatrix().toArray()
#     df_confusion_matrix = pd.DataFrame(confusion_matrix, columns=['Predicted Not Churned', 'Predicted Churned'])
#     df_confusion_matrix.index = ['Actual Not Churned', 'Actual Churned']
    
#     # Calculate confusion matrix
#     predictionsAndLabels = colSelect.rdd.map(lambda x: (x.prediction, x.label))
#     metrics = MulticlassMetrics(predictionsAndLabels)
#     confusionMatrix = metrics.confusionMatrix().toArray()

    
    # Create results dictionary
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        #'confusionMatrix': print(df_confusion_matrix),
        'auc': auc,
        'pr_curve': pr_curve
        #'classification_report': classification_report
    }

    return results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
trainingTransformed.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['userId', 'gender', 'churn_cancellation', 'days_registered', 'avg_songs', 'last_level', 'thumbs_up_count', 'thumbs_down_count', 'num_friends', 'avg_roll_advert', 'help_visit', 'error_visit', 'total_playlist', 'features', 'normalized_features']

In [32]:
#run tikll here

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
testTransformed.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['userId', 'gender', 'churn_cancellation', 'days_registered', 'avg_songs', 'last_level', 'thumbs_up_count', 'thumbs_down_count', 'num_friends', 'avg_roll_advert', 'help_visit', 'error_visit', 'total_playlist', 'features', 'normalized_features']

In [34]:
#trainingTransformed.rdd.saveAsPickleFile("trainingTransformed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#testTransformed.rdd.saveAsPickleFile("testTransformed")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Random Forest

In [48]:
rf = RandomForestClassifier(labelCol="churn_cancellation",
                            featuresCol="features", 
                            maxDepth=10, 
                            numTrees=100, 
                            seed=23)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
modelrf = rf.fit(trainingTransformed)
pred_rf = modelrf.transform(trainingTransformed)
pred_test_rf = modelrf.transform(testTransformed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-49:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3667



In [50]:
evaluation_results_rf = ModelEvaluator(pred_test_rf, label='churn_cancellation')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-50:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 4073



In [57]:
evaluation_results_rf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'accuracy': 0.7880818692265894, 'precision': 0.6703126902226763, 'recall': 0.7886252490490854, 'f1': 0.7233838415811581, 'auc': 0.4835815732939627, 'pr_curve': 0.10447654776436616}

### Logistics Regression

In [52]:
lr = LogisticRegression(labelCol="churn_cancellation",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.01,
                        elasticNetParam=0.8,
                        family="binomial",
                        threshold=0.5,
                        fitIntercept=True,
                        standardization=True,
                        aggregationDepth=2)

modellr = lr.fit(trainingTransformed)
pred_lr = modellr.transform(trainingTransformed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-52:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 4378



In [53]:
pred_test_lr = modellr.transform(testTransformed)
evaluation_results_lr = ModelEvaluator(pred_test_lr, label='churn_cancellation')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
evaluation_results_lr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'accuracy': 0.8185111392863612, 'precision': 0.7526452749151896, 'recall': 0.8185111392863612, 'f1': 0.7407019782290581, 'auc': 0.5036675713666864, 'pr_curve': 0.3302220459179602}

### GBT

In [59]:
from pyspark.ml.classification import GBTClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
gbt = GBTClassifier(featuresCol="normalized_features", labelCol="churn_cancellation", maxIter=10, maxBins=820)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
modelgbt = gbt.fit(trainingTransformed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-61:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 5162



In [62]:
pred_gbt = modelgbt.transform(trainingTransformed)
pred_test_gbt = modelgbt.transform(testTransformed)
evaluation_results_gbt = ModelEvaluator(pred_test_gbt, label='churn_cancellation')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-62:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 6571



In [63]:
evaluation_results_gbt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'accuracy': 0.8134395942763992, 'precision': 0.7267936140059392, 'recall': 0.8127150878464046, 'f1': 0.7416445739068951, 'auc': 0.5055720827844722, 'pr_curve': 0.24207675135948997}

In [ ]:
# best fit model - random forest!

In [37]:
rf = RandomForestClassifier()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [25, 50, 100]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()
    #.addGrid(rf.impurity, ["gini", "entropy"]) \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
data = trainingTransformed.select("normalized_features","churn_cancellation")
data = data.withColumnRenamed('normalized_features', 'features')
data = data.withColumnRenamed('churn_cancellation', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
cvModel = crossval.fit(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-41:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 9061



In [42]:
bestModel_rf = cvModel.bestModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
pred_rf_cv = bestModel_rf.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
results = ModelEvaluator(pred_rf_cv,"label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'accuracy': 0.8161220644331081, 'precision': 0.665980211500173, 'recall': 0.8161220644331081, 'f1': 0.7335380993806615, 'auc': 0.5, 'pr_curve': 0.5920639053146803}

In [48]:
data = testTransformed.select("normalized_features","churn_cancellation")
data = data.withColumnRenamed('normalized_features', 'features')
data = data.withColumnRenamed('churn_cancellation', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
pred_test_rf_cv = bestModel_rf.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
results = ModelEvaluator(pred_test_rf_cv,"label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-52:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 14454



In [53]:
results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'accuracy': 0.8185111392863612, 'precision': 0.6702301357924552, 'recall': 0.8185111392863612, 'f1': 0.7370757975978979, 'auc': 0.4998893805309734, 'pr_curve': 0.18130773410614018}

In [54]:
# Brest fit ,model - logistic regression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
lr=LogisticRegression()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.5, 1.0]) \
    .addGrid(lr.maxIter, [10, 100]) \
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
data = trainingTransformed.select("normalized_features","churn_cancellation")
data = data.withColumnRenamed('normalized_features', 'features')
data = data.withColumnRenamed('churn_cancellation', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
cvModel_lr = crossval.fit(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-64:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 15144



In [65]:
bestModel_lr = cvModel_lr.bestModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-65:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [66]:
pred_lr_cv = bestModel_lr.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-66:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [ ]:
#results = ModelEvaluator(pred_lr_cv,"label")

In [69]:
data = testTransformed.select("normalized_features","churn_cancellation")
data = data.withColumnRenamed('normalized_features', 'features')
data = data.withColumnRenamed('churn_cancellation', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-69:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [70]:
predtest_lr_cv = bestModel_lr.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-70:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [71]:
results = ModelEvaluator(predtest_lr_cv,"label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-71:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [72]:
results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'accuracy': 0.817062126426372, 'precision': 0.7555168679114315, 'recall': 0.8174243796413693, 'f1': 0.7480421698218961, 'auc': 0.5122263798148754, 'pr_curve': 0.31088406446131495}

Exception in thread cell_monitor-72:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'

